In [24]:
import numpy as np
import csv
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService
from quantum_serverless import QuantumServerless, ServerlessProvider, Program
import os
import time
import psutil
import networkx as nx
from qiskit.opflow import PauliSumOp
from qiskit.quantum_info import SparsePauliOp, Pauli

def create_maxcut_hamiltonian(num_qubits):
    # Create a fully connected graph as an example
    G = nx.complete_graph(num_qubits)
    
    # Initialize an empty list to store Pauli terms
    pauli_terms = []

    for i, j in G.edges():
        # Construct the Pauli string
        pauli_str = ['I'] * num_qubits
        pauli_str[i] = 'Z'
        pauli_str[j] = 'Z'

        # Convert to SparsePauliOp term
        sparse_term = SparsePauliOp(Pauli(''.join(pauli_str)), [0.5])
        pauli_terms.append(sparse_term)

    # Sum all the SparsePauliOp terms
    sparse_hamiltonian = sum(pauli_terms)

    return PauliSumOp(sparse_hamiltonian)


# Function to get CPU and memory usage
def get_cpu_memory_usage():
    cpu_percent = psutil.cpu_percent()
    mem_percent = psutil.virtual_memory().percent
    return cpu_percent, mem_percent

# Function to calculate mean CPU and memory usage
def calculate_mean_usage():
    cpu_values = []  # List to store CPU usage values
    mem_values = []  # List to store memory usage values

    for _ in range(5):  # Adjust the number of samples as needed
        cpu_usage, mem_usage = get_cpu_memory_usage()
        cpu_values.append(cpu_usage)
        mem_values.append(mem_usage)
        time.sleep(2)

    mean_cpu = sum(cpu_values) / len(cpu_values)
    mean_mem = sum(mem_values) / len(mem_values)

    return mean_cpu, mean_mem

# Function to run QAOA for a given ansatz and save the results
def run_and_save_qaoa(ansatz, num_qubits, num_layers, num_iterations, operator, csv_writer):
    USE_RUNTIME_SERVICE = False

    service = None
    backend = "ibmq_qasm_simulator"
    if USE_RUNTIME_SERVICE:
        service = QiskitRuntimeService()

    input_arguments = {
        "initial_point": None,
        "ansatz": ansatz,
        "operator": operator,
        "service": service,
        "backend": backend,
        "iterations": num_iterations  # Added iterations to the arguments
    }

    provider = ServerlessProvider(
       username=os.environ.get("GATEWAY_USER", "user"),
        password=os.environ.get("GATEWAY_PASSWORD", "password123"),
        host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    )

    serverless = QuantumServerless(provider)
    
    program = Program(
        title=f"QAOA_{num_qubits}_qubits_{num_layers}_layers_{num_iterations}_iterations",
        entrypoint="program.py",
        working_dir="./",
    )
    
     # Draw and print the quantum circuit
    print(f"\nQuantum Circuit for {num_qubits} Qubits and {num_layers} Layers:")
    print(ansatz.decompose().draw())  # Draw the circuit


    # Measure execution time
    start_time = time.time()

    job = serverless.run(program, arguments=input_arguments)

    while job.status() == 'QUEUED' or job.status() == 'RUNNING':
        time.sleep(2)

    result = job.result()
    optimal_point = result.get("optimal_point")
    optimal_value = result.get("optimal_value")

    # Calculate mean CPU and memory usage
    mean_cpu, mean_mem = calculate_mean_usage()

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Write results to CSV
    csv_writer.writerow({
        "Num_Qubits": num_qubits,
        "Layers": num_layers,
        "Iterations": num_iterations,
        "Optimal_Point": optimal_point,
        "Optimal_Value": optimal_value,
        "Mean_CPU": mean_cpu,
        "Mean_Memory": mean_mem,
        "Elapsed_Time": elapsed_time
    })

    print(f"QAOA for {num_qubits} qubits, {num_layers} layers, {num_iterations} iterations completed.")
    print(f"Elapsed Time: {elapsed_time} seconds")
    print(f"Mean CPU Usage: {mean_cpu}%")
    print(f"Mean Memory Usage: {mean_mem}%")

# Create a CSV file to save the results
csv_file_path = "metrics.csv"
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ["Num_Qubits", "Layers", "Iterations", "Optimal_Point", "Optimal_Value", "Mean_CPU", "Mean_Memory", "Elapsed_Time"]
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    num_qubits_list = [2, 4, 6]   # List of qubit numbers to test
    layers_list = [1, 2, 4, 6]    # List of layers to test
    iterations_list = [10, 50, 100]  # List of iteration numbers to test

    for num_qubits in num_qubits_list:
      
        operator = create_maxcut_hamiltonian(num_qubits)

        for num_layers in layers_list:
            ansatz = QAOAAnsatz(operator, reps=num_layers)

            for num_iterations in iterations_list:
                run_and_save_qaoa(ansatz, num_qubits, num_layers, num_iterations, operator, csv_writer)

print(f"Results saved in {csv_file_path}.")


/var/folders/xx/lh8d8jbs62ndd23zycn13qj00000gn/T/ipykernel_5311/4098967137.py:34: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  return PauliSumOp(sparse_hamiltonian)
Attempting to instrument while already instrumented



Quantum Circuit for 2 Qubits and 1 Layers:
     ┌───┐┌────────────────────────┐┌───────────────────────────┐
q_0: ┤ H ├┤0                       ├┤0                          ├
     ├───┤│  exp(-it ZZ)(1.0*γ[0]) ││  exp(-it (XI + IX))(β[0]) │
q_1: ┤ H ├┤1                       ├┤1                          ├
     └───┘└────────────────────────┘└───────────────────────────┘
QAOA for 2 qubits, 1 layers, 10 iterations completed.
Elapsed Time: 30.765668869018555 seconds
Mean CPU Usage: 31.620000000000005%
Mean Memory Usage: 80.82000000000001%


Attempting to instrument while already instrumented



Quantum Circuit for 2 Qubits and 1 Layers:
     ┌───┐┌────────────────────────┐┌───────────────────────────┐
q_0: ┤ H ├┤0                       ├┤0                          ├
     ├───┤│  exp(-it ZZ)(1.0*γ[0]) ││  exp(-it (XI + IX))(β[0]) │
q_1: ┤ H ├┤1                       ├┤1                          ├
     └───┘└────────────────────────┘└───────────────────────────┘
QAOA for 2 qubits, 1 layers, 50 iterations completed.
Elapsed Time: 24.547974824905396 seconds
Mean CPU Usage: 39.36%
Mean Memory Usage: 81.17999999999999%


Attempting to instrument while already instrumented


...


Quantum Circuit for 6 Qubits and 6 Layers:
     ┌───┐»
q_0: ┤ H ├»
     ├───┤»
q_1: ┤ H ├»
     ├───┤»
q_2: ┤ H ├»
     ├───┤»
q_3: ┤ H ├»
     ├───┤»
q_4: ┤ H ├»
     ├───┤»
q_5: ┤ H ├»
     └───┘»
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                                                                                           ├»
«     │                                                                                                                                                            │»
«q_1: ┤1                                                                                                                                                           ├»
«     │                                                                                                                                 

Attempting to instrument while already instrumented



Quantum Circuit for 6 Qubits and 6 Layers:
     ┌───┐»
q_0: ┤ H ├»
     ├───┤»
q_1: ┤ H ├»
     ├───┤»
q_2: ┤ H ├»
     ├───┤»
q_3: ┤ H ├»
     ├───┤»
q_4: ┤ H ├»
     ├───┤»
q_5: ┤ H ├»
     └───┘»
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                                                                                           ├»
«     │                                                                                                                                                            │»
«q_1: ┤1                                                                                                                                                           ├»
«     │                                                                                                                                 

Attempting to instrument while already instrumented



Quantum Circuit for 6 Qubits and 6 Layers:
     ┌───┐»
q_0: ┤ H ├»
     ├───┤»
q_1: ┤ H ├»
     ├───┤»
q_2: ┤ H ├»
     ├───┤»
q_3: ┤ H ├»
     ├───┤»
q_4: ┤ H ├»
     ├───┤»
q_5: ┤ H ├»
     └───┘»
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                                                                                           ├»
«     │                                                                                                                                                            │»
«q_1: ┤1                                                                                                                                                           ├»
«     │                                                                                                                                 

In [4]:
ansatz.draw(fold=-1)

┌────────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                   ├
     │                                                                    │
q_1: ┤1                                                                   ├
     │                                                                    │
q_2: ┤2 QAOA(γ[0],β[0],γ[1],β[1],γ[2],β[2],γ[3],β[3],γ[4],β[4],γ[5],β[5]) ├
     │                                                                    │
q_3: ┤3                                                                   ├
     │                                                                    │
q_4: ┤4                                                                   ├
     └────────────────────────────────────────────────────────────────────┘